In [1]:
import pandas as pd

Check on excessive zeros and verify original data remains uncorrupted from raw original. 


In [18]:
eda_df = pd.read_hdf(r'C:\Users\Linds\Repos\East_River\data\processed\east_river_engineered_features.h5')

In [19]:
org_scada_df = pd.read_csv(r'C:\Users\Linds\Repos\East_River\data\processed\Cleaned_SCADA_Data.csv')

In [32]:
org_weather_df = pd.read_json(r'C:\Users\Linds\Repos\East_River\data\processed\time_processed_weather.json')

In [20]:
shared_cols = eda_df.columns.intersection(org_weather_df.columns)
print("Shared columns between eda_df and org_weather_df:")
print(shared_cols.tolist())

Shared columns between eda_df and org_weather_df:
['location', 'datetime', 'temperature', 'wind_speed', 'wind_degree', 'precip', 'humidity', 'visibility', 'pressure', 'cloudcover', 'heatindex', 'windchill', 'windgust', 'feelslike', 'uv_index', 'obs_tms_lcl', 'wind_dir', 'weather_descriptions']


In [21]:
shared_cols_scada = eda_df.columns.intersection(org_scada_df.columns)
print("Shared columns between eda_df and org_scada_df:")
print(shared_cols_scada.tolist())

Shared columns between eda_df and org_scada_df:
['Unnamed: 0', 'Timestamp', 'OnLine_Load_MW', 'Load_Control_KW', 'Load_Control_MW', 'Estimated_Online_Load_Control_MW', 'Control_Threshold_MW']


In [22]:
# ensure that the Timestamp columns are in the same datetime format/timezone
org_scada_df['Timestamp'] = pd.to_datetime(org_scada_df['Timestamp']).dt.tz_localize('UTC')
eda_df['Timestamp'] = pd.to_datetime(eda_df['Timestamp'])

# merge the two dataframes on 'Timestamp' using only the shared SCADA columns from eda_df
common_cols = ['Timestamp', 'OnLine_Load_MW', 'Load_Control_KW', 'Load_Control_MW',
               'Estimated_Online_Load_Control_MW', 'Control_Threshold_MW']
merged_df = org_scada_df.merge(eda_df[common_cols], on='Timestamp', suffixes=('_scada', '_eda'))

# compare each column from SCADA with the corresponding one from eda_df
cols_to_check = ['OnLine_Load_MW', 'Load_Control_KW', 'Load_Control_MW',
                 'Estimated_Online_Load_Control_MW', 'Control_Threshold_MW']

for col in cols_to_check:
    same = merged_df[f"{col}_scada"].equals(merged_df[f"{col}_eda"])
    print(f"{col} matches:", same)

OnLine_Load_MW matches: True
Load_Control_KW matches: True
Load_Control_MW matches: True
Estimated_Online_Load_Control_MW matches: True
Control_Threshold_MW matches: True


In [23]:
cols_to_check = ['OnLine_Load_MW', 'Load_Control_KW', 'Load_Control_MW',
                 'Estimated_Online_Load_Control_MW', 'Control_Threshold_MW']

for col in cols_to_check:
    full_match = merged_df[f"{col}_scada"].equals(merged_df[f"{col}_eda"])
    print(f"{col} full match: {full_match}")

OnLine_Load_MW full match: True
Load_Control_KW full match: True
Load_Control_MW full match: True
Estimated_Online_Load_Control_MW full match: True
Control_Threshold_MW full match: True


In [26]:
eda_df = eda_df.sort_values(['location', 'datetime'])

In [27]:
eda_df.head()

,location,datetime,temperature,wind_speed,wind_degree,precip,humidity,visibility,pressure,cloudcover,...,day_of_year,day_of_year_sin,day_of_year_cos,local_date,prev_day_peak,prev_day_avg,iso_week,prev_week_peak,temp_humidity_interaction,wind_precip_interaction
0,"Aberdeen, South Dakota",2020-12-31 06:00:00+00:00,12.0,7.0,197.0,0.0,90.0,6.0,30.120,69.0,...,366,0.017213,0.999852,2020-12-31,NaN,NaN,53,NaN,1080.0,0.0
225,"Aberdeen, South Dakota",2020-12-31 06:30:00+00:00,11.0,7.0,196.0,0.0,90.0,6.0,30.105,61.0,...,366,0.017213,0.999852,2020-12-31,NaN,NaN,53,NaN,990.0,0.0
441,"Aberdeen, South Dakota",2020-12-31 07:00:00+00:00,10.0,7.0,195.0,0.0,90.0,6.0,30.090,53.0,...,366,0.017213,0.999852,2020-12-31,NaN,NaN,53,NaN,900.0,0.0
660,"Aberdeen, South Dakota",2020-12-31 07:30:00+00:00,10.0,7.0,194.0,0.0,89.5,6.0,30.090,45.5,...,366,0.017213,0.999852,2020-12-31,NaN,NaN,53,NaN,895.0,0.0
763,"Aberdeen, South Dakota",2020-12-31 08:00:00+00:00,10.0,7.0,193.0,0.0,89.0,6.0,30.090,38.0,...,366,0.017213,0.999852,2020-12-31,NaN,NaN,53,NaN,890.0,0.0


In [33]:
org_weather_df.head()

,location,datetime,temperature,wind_speed,wind_degree,precip,humidity,visibility,pressure,cloudcover,heatindex,dewpoint,windchill,windgust,feelslike,uv_index,obs_tms_lcl,wind_dir,weather_descriptions
0,"Aberdeen, South Dakota",2020-12-31 06:00:00+00:00,12.0,7.0,197.0,0.0,90.0,6.0,30.120,69.0,12.0,9.0,0.0,15.0,0.0,1.0,2020-12-31T06:00:00.000Z,SSW,Cloudy
1,"Aberdeen, South Dakota",2020-12-31 06:30:00+00:00,11.0,7.0,196.0,0.0,90.0,6.0,30.105,61.0,11.0,9.0,0.0,15.5,0.0,1.0,2020-12-31T06:00:00.000Z,SSW,Cloudy
2,"Aberdeen, South Dakota",2020-12-31 07:00:00+00:00,10.0,7.0,195.0,0.0,90.0,6.0,30.090,53.0,10.0,9.0,0.0,16.0,0.0,1.0,2020-12-31T07:00:00.000Z,SSW,Partly cloudy
3,"Aberdeen, South Dakota",2020-12-31 07:30:00+00:00,10.0,7.0,194.0,0.0,89.5,6.0,30.090,45.5,10.0,9.0,0.0,16.0,0.0,1.0,2020-12-31T07:00:00.000Z,SSW,Partly cloudy
4,"Aberdeen, South Dakota",2020-12-31 08:00:00+00:00,10.0,7.0,193.0,0.0,89.0,6.0,30.090,38.0,10.0,9.0,0.0,16.0,0.0,1.0,2020-12-31T08:00:00.000Z,SSW,Partly cloudy


Missing Values

In [38]:
print(eda_df.isnull().sum().to_string())

location                                  0
datetime                                  0
temperature                               0
wind_speed                                0
wind_degree                               0
precip                                    0
humidity                                  0
visibility                                0
pressure                                  0
cloudcover                                0
heatindex                                 0
windchill                                 0
windgust                                  0
feelslike                                 0
uv_index                                  0
obs_tms_lcl                               0
wind_dir                                  0
weather_descriptions                      0
temperature_2m                            0
precipitation                             0
snowfall                                  0
snow_depth                                0
apparent_temperature            

Zero Values

In [41]:
zero_counts = (eda_df == 0).sum().to_string()
print("Zero counts per column in eda_df:")
print(zero_counts)

Zero counts per column in eda_df:
location                                   0
datetime                                   0
temperature                            40220
wind_speed                              1203
wind_degree                             4465
precip                              11896377
humidity                                   0
visibility                            262927
pressure                                   0
cloudcover                            835753
heatindex                              40227
windchill                              64114
windgust                                 264
feelslike                              64114
uv_index                                   0
obs_tms_lcl                                0
wind_dir                                   0
weather_descriptions                       0
temperature_2m                         31494
precipitation                       11086236
snowfall                            11778408
snow_depth           